<a href="https://colab.research.google.com/github/qdtruong/Public/blob/main/SAMgovWageDetermination_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install dependencies
%pip install -q google-colab-selenium

#import libraries
import google_colab_selenium as gs
import time
import re
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
from datetime import datetime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [2]:
# Create list of all 50 state abbreviations plus Washington, D.C.
state_abbreviations = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY',
    'DC'
]

# Create Master Dataframe/Names Dataframe
dfMaster = pd.DataFrame(columns=['search','href'])
dfNames = pd.DataFrame(columns=['search','wd num','state', 'county', 'date', 'revision_num'])

#navigate to Sam.gov website
driver = gs.Chrome()

for index, state_abbreviations in enumerate(state_abbreviations):
    print(f"{index}: {state_abbreviations}")
    url = f"https://sam.gov/search/?page=1&pageSize=100&sort=&index=sca&sfm%5Bstatus%5D%5Bis_active%5D=true&sfm%5BwdLocationWrapper%5D%5BwdStates%5D%5B0%5D%5Bkey%5D={state_abbreviations}&sfm%5BwdLocationWrapper%5D%5BwdStates%5D%5B0%5D%5Bvalue%5D=District%20of%20Columbia"

    #Scrape HTML
    driver.get(url)
    time.sleep(1)
    html = driver.page_source

    #Parse HTML
    soup = BeautifulSoup(html)
    wdsite = []

    #data for the Master Dataframe
    for i in soup.find_all('a', href=re.compile("wage")):
        wdsite.append(i['href'])

    dfAppend = pd.DataFrame({'search': f'{state_abbreviations}','href': wdsite,})
    dfMaster = pd.concat([dfMaster, dfAppend], ignore_index=True)

    #data for the Names Dataframe
    wd_elements = soup.find_all('div', class_ = {"sds-field__value","sds-field__name", "grid-col"})
    wd_list = [tag.text for tag in wd_elements]
    wd_string = '\n'.join(wd_list)

    # Regular expression pattern to match each chunk
    pattern = r"(Service Contract Act WD #:.*?Published Date\n.*?\d{4})"

    # Find all chunks using the pattern
    chunks = re.findall(pattern, wd_string, re.DOTALL)

    # Initialize list to store data
    rows = []

    # Process each chunk to extract information
    for chunk in chunks:
        # Extract WD number
        wd_number_match = re.search(r"Service Contract Act WD #:\s*(\S+)", chunk)
        wd_number = wd_number_match.group(1) if wd_number_match else None

        # Extract published date
        published_date_match = re.search(r"Published Date\n(.*)", chunk)
        published_date = published_date_match.group(1).strip() if published_date_match else None
        date_obj = datetime.strptime(published_date, '%b %d, %Y')

        # Extract revision number
        revision_number_match = re.search(r"Revision Number\n(\d+)", chunk)
        revision_num = revision_number_match.group(1) if revision_number_match else None

        # Extract states and counties
        states_and_counties = re.findall(r"State\n(.*?)\nCounties\n(.*?)\n", chunk, re.DOTALL)
        for state, counties in states_and_counties:
            state_name = state.strip()
            county_names = counties.strip()  # Get the full counties line as a single string

            rows.append({'search':f'{state_abbreviations}', 'wd num': wd_number, 'state': state_name, 'county': county_names, 'date': date_obj, 'revision_num': revision_num })

    # Create a DataFrame from the list of dictionaries
    dfAppend2 = pd.DataFrame(rows)
    dfNames = pd.concat([dfNames, dfAppend2], ignore_index=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0: AL


<ipython-input-2-3ca6cfb889f0>:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfNames = pd.concat([dfNames, dfAppend2], ignore_index=True)


1: AK
2: AZ
3: AR
4: CA
5: CO
6: CT
7: DE
8: FL
9: GA
10: HI
11: ID
12: IL
13: IN
14: IA
15: KS
16: KY
17: LA
18: ME
19: MD
20: MA
21: MI
22: MN
23: MS
24: MO
25: MT
26: NE
27: NV
28: NH
29: NJ
30: NM
31: NY
32: NC
33: ND
34: OH
35: OK
36: OR
37: PA
38: RI
39: SC
40: SD
41: TN
42: TX
43: UT
44: VT
45: VA
46: WA
47: WV
48: WI
49: WY
50: DC


In [3]:
# Transform href column into usable hyperlink where the download is

def transform_url(value):
    base_url = "https://sam.gov/api/prod/wdol/v1/wd"
    parts = value.split('/')
    transformed = f"{base_url}/{parts[2]}/{parts[3]}/download?"
    return transformed

# Apply transformation to create a new columns
dfMaster['download URL'] = dfMaster['href'].apply(transform_url)
dfMaster['wd num'] = dfMaster['href'].str.split('/').str[2]

# De-duplicate/Clean up dfNames
abbreviation_to_state = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia'
}
# Trim whitespace and replace double spaces in all columns
dfNames = dfNames.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)

# Adding the state names to the DataFrame so dfNames doesn't introduce duplicates on the merge
dfNames['State Name'] = dfNames['search'].map(abbreviation_to_state)
# without this filter, a WD with multiple states listed on the same search (DC=>Maryland, Virginia,DC) will create dupes
dfNames = dfNames[dfNames['state'] == dfNames['State Name']]

dfNamesMerged = pd.merge(dfMaster, dfNames, on=['search', 'wd num'], how='left')


<ipython-input-3-f6fae437a064>:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfNames = dfNames.applymap(lambda x: ' '.join(x.split()) if isinstance(x, str) else x)


In [4]:
# New England WDs are missing because the specific WDs in those 6 states are written differently
# this code will run another search for just those pages

state_abbreviations = [ 'CT', 'MA', 'ME', 'RI', 'NH', 'VT']
dfNewEngland = pd.DataFrame(columns=['search','wd num','state', 'county', 'date', 'revision_num'])

for index, state_abbreviations in enumerate(state_abbreviations):
    print(f"{index}: {state_abbreviations}")
    url = f"https://sam.gov/search/?page=1&pageSize=100&sort=&index=sca&sfm%5Bstatus%5D%5Bis_active%5D=true&sfm%5BwdLocationWrapper%5D%5BwdStates%5D%5B0%5D%5Bkey%5D={state_abbreviations}&sfm%5BwdLocationWrapper%5D%5BwdStates%5D%5B0%5D%5Bvalue%5D=District%20of%20Columbia"

    #Scrape HTML
    driver.get(url)
    time.sleep(1)
    html = driver.page_source

    #Parse HTML
    soup = BeautifulSoup(html)

    ##########
    wd_elements = soup.find_all('div', class_ = {"sds-field__value","sds-field__name", "grid-col"})
    wd_list = [tag.text for tag in wd_elements]
    data = '\n'.join(wd_list)

    # Regular expression pattern to match each chunk
    pattern = r"(Service Contract Act WD #:.*?Published Date\n.*?\d{4})"

    # Find all chunks using the pattern
    chunks = re.findall(pattern, data, re.DOTALL)

    # Initialize list to store data
    rows = []

    # Process each chunk to extract information
    for chunk in chunks:

        # Extract WD number
        wd_number_match = re.search(r"Service Contract Act WD #:\s*(\S+)", chunk)
        wd_number = wd_number_match.group(1) if wd_number_match else None

        # Extract published date
        published_date_match = re.search(r"Published Date\n(.*)", chunk)
        published_date = published_date_match.group(1).strip() if published_date_match else None
        date_obj = datetime.strptime(published_date, '%b %d, %Y')

        # Extract revision number
        revision_number_match = re.search(r"Revision Number\n(\d+)", chunk)
        revision_num = revision_number_match.group(1) if revision_number_match else None

        # Split text using the delimiters ":" and "," and "\n" and the word "in" and the sudden capitalized letters in the middle of location_description
        location_description = re.split(r'[:,\n-]|(?<=[a-z])(?=[A-Z]| in )', chunk)

        # Remove any leading/trailing whitespace from each item
        location_description = [word.strip() for word in location_description if word.strip()]

        new_england = ['Connecticut', 'Maine','Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont']
        state_pattern = re.compile(r'\b(' + '|'.join(new_england) + r')\b', re.IGNORECASE)
        ne_rows = []

        for i in range(len(location_description)):
            line = location_description[i].strip()

            if state_pattern.search(line):
                state = state_pattern.search(line).group(0)
            elif re.search(r'county', line, re.IGNORECASE):
                county = location_description[i]
                county = line.replace("in ", "").replace(".", "")
                county = re.sub(r'\bcounty\b', '', county, flags=re.IGNORECASE).strip()
                ne_rows.append({'search':state_abbreviations, 'state':state, 'county': county, 'wd num':wd_number, 'date':date_obj, 'revision_num':revision_num})

        dfAppend3 = pd.DataFrame(ne_rows)
        dfNewEngland = pd.concat([dfNewEngland, dfAppend3], ignore_index=True)

        dfNewEngland = dfNewEngland.groupby(['wd num', 'state', 'revision_num'], as_index=False).agg({'county': ', '.join,
        'search': 'first','date': 'first' })

#tidying up the state column in dfNewEngland
dfNewEngland['state'] = dfNewEngland['state'].str.title()

#for some reason the 'search' field is giving wrong state abbreviations for some rows...this fixes that
state_to_abbreviation = {state: abbreviation for abbreviation, state in abbreviation_to_state.items()}
dfNewEngland['search'] = dfNewEngland['state'].map(state_to_abbreviation)

#deduping dfNewEngland
dfNewEngland['State Name'] = dfNewEngland['search'].map(abbreviation_to_state)
dfNewEngland = dfNewEngland[dfNewEngland['state'] == dfNewEngland['State Name']] #technically this line is not needed

# Impute missing values
dfMasterFinal = dfNamesMerged.merge(dfNewEngland, on=['search', 'wd num'], suffixes=('_df1', '_df2'), how='left')
dfMasterFinal['state'] = dfMasterFinal['state_df1'].combine_first(dfMasterFinal['state_df2'])
dfMasterFinal['county'] = dfMasterFinal['county_df1'].combine_first(dfMasterFinal['county_df2'])
dfMasterFinal['date'] = dfMasterFinal['date_df1'].combine_first(dfMasterFinal['date_df2'])
dfMasterFinal['revision_num'] = dfMasterFinal['revision_num_df1'].combine_first(dfMasterFinal['revision_num_df2'])
dfMasterFinal['State Name'] = dfMasterFinal['State Name_df1'].combine_first(dfMasterFinal['State Name_df2'])

# Select the relevant columns to create the final imputed DataFrame
dfMasterFinal = dfMasterFinal[['search','wd num','href','download URL','state', 'county', 'date', 'revision_num']]



0: CT


<ipython-input-4-3bd64d12084f>:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfNewEngland = pd.concat([dfNewEngland, dfAppend3], ignore_index=True)


1: MA
2: ME
3: RI
4: NH
5: VT


In [12]:
# shorten length of County column to ensure filename is not too long
max_length = 180

def shorten_string(text):
    if len(text) <= max_length:
        return text

    # Truncate to the max length
    truncated = text[:max_length]

    # Find the last comma before the truncation point
    last_comma = truncated.rfind(',')
    if last_comma != -1:
        truncated = truncated[:last_comma]

    return truncated

# Apply the function to each row in the 'long_strings' column
dfMasterFinal['county_shortened'] = dfMasterFinal['county'].apply(shorten_string)

In [14]:
# Download all the files on the dfMasterFinal, putting them into a folder named after the State Abbreviation of the search

# Paths
desktop_path = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop')
sam_folder_path = os.path.join(desktop_path, 'SAM')
if not os.path.exists(sam_folder_path):
    os.makedirs(sam_folder_path)

dfFails = pd.DataFrame(columns=dfMasterFinal.columns)

for i, row in dfMasterFinal.iterrows():
    url = row['download URL']
    filename = f"{row['wd num']}, Rev {row['revision_num']} [{row['search']}] county of {row['county_shortened']}"
    subdirectory = f"{row['state']} ({row['search']})"

    sub_folder_path = os.path.join(sam_folder_path, f'{subdirectory}')
    if not os.path.exists(sub_folder_path):
        os.makedirs(sub_folder_path)

    # Path to save the downloaded file
    file_path = os.path.join(sam_folder_path, f'{subdirectory}/{filename}.txt')

    # Download the file
    response = requests.get(url)
    time.sleep(1)

    # Check if the request was successful
    if response.status_code == 200:
        # Save the file to the specified path
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"File saved successfully at {file_path}")
    else:
        print(f"Failed to download the file. {filename} Status code: {response.status_code}")
        dfAppend = pd.DataFrame({'search': subdirectory,'href': wdsite,'download URL': url,'filename': filename,})
        dfFails = pd.concat([dfFails, dfAppend], ignore_index=True)

File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4597, Rev 26 [AL] county of Geneva, Henry, Houston.txt
File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4631, Rev 27 [AL] county of Barbour, Coffee, Dale.txt
File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4615, Rev 27 [AL] county of Cherokee, Clay, Cleburne, De Kalb, Talladega.txt
File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4599, Rev 26 [AL] county of Colbert, Lauderdale.txt
File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4629, Rev 27 [AL] county of Sumter.txt
File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4613, Rev 27 [AL] county of Franklin, Marion, Winston.txt
File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4593, Rev 27 [AL] county of Chilton.txt
File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4619, Rev 27 [AL] county of Coosa, Tallapoosa.txt
File saved successfully at /root/Desktop/SAM/Alabama (AL)/2015-4623, Re

In [16]:
#dfMaster.to_csv('SAMdata.csv')

!zip -r /root/Desktop/"SAM.gov Wage Determination".zip /root/Desktop/SAM
from google.colab import files
files.download("/root/Desktop/SAM.gov Wage Determination.zip")

#other notes:
#1 do they want duplicates when a file can be found in different states?
#2 do they REALLY want PDFs? that seems like a terrible idea
#3 is 908 the actual number of files in all 51 combined searches (I've gotten different numbers when I get rid of the delay)
#4 it would be nice to include more data into the dfMaster (have a column for state, county, revision #, etc. Would also be nice to have the documents on the dataframe as a list/dictionary so it can be searched)
#5 some files do not download from GA, IL, IA, KY, NM, OH, OK, TN, TX, WI,
#6 check that the files are winding up in the correct state directories (ah, it happens because the code basically does a vlookup...it would be better to use index numbers...though this way I think de-duplicates it)
#7 total time to execute downloads is 18m

  adding: root/Desktop/SAM/ (stored 0%)
  adding: root/Desktop/SAM/Louisiana (LA)/ (stored 0%)
  adding: root/Desktop/SAM/Louisiana (LA)/2015-5833, Rev 25 [LA] county of Iberia.txt (deflated 75%)
  adding: root/Desktop/SAM/Louisiana (LA)/2015-5209, Rev 26 [LA] county of Jefferson Davis.txt (deflated 75%)
  adding: root/Desktop/SAM/Louisiana (LA)/2015-5195, Rev 23 [LA] county of Washington.txt (deflated 75%)
  adding: root/Desktop/SAM/Louisiana (LA)/2015-5759, Rev 24 [LA] county of Acadia.txt (deflated 75%)
  adding: root/Desktop/SAM/Louisiana (LA)/2015-5175, Rev 25 [LA] county of Grant, Rapides.txt (deflated 75%)
  adding: root/Desktop/SAM/Louisiana (LA)/2015-5185, Rev 26 [LA] county of Calcasieu, Cameron.txt (deflated 75%)
  adding: root/Desktop/SAM/Louisiana (LA)/2015-5177, Rev 25 [LA] county of Ascension, East Baton Rouge, East Feliciana, Iberville, Livingston, Pointe Coupee, St Helena, West Baton Rouge, West Feliciana.txt (deflated 75%)
  adding: root/Desktop/SAM/Louisiana (LA)/201

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
from google.colab import files
dfMasterFinal.to_csv('SAMdata.csv')
files.download('SAMdata.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>